In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from deep4deep.w2v_embedding_and_rnn_model import Preprocessor

In [4]:
from deep4deep import trainer
#comment éviter de recharger le glove à chaque fois? 
trainer.demo()

initializing Trainer
downloading transfer learning embedder – this may take time, coffee break maybe?
> /Users/admin/code/ClemenceK/deep4deep/deep4deep/trainer.py(53)merge_df()
     52     import ipdb; ipdb.set_trace()
---> 53     return df_from_main_module.join(my_df_meta_categorical, on='id', how='inner', )
     54 

ipdb> exit()


BdbQuit: 

In [ ]:
from os import path
path.join(path.dirname(path.dirname(__file__)), "raw_data", "data2020-12-03.csv")